In [1]:
import numpy as np
import pandas as pd
import catboost as cat
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
 
pd.set_option('precision', 5)
pd.set_option('display.float_format', lambda x: '%.5f' % x) 
pd.options.display.max_rows = 200

In [2]:
train_df = pd.read_csv('data/train/train_trd.csv')
test_df = pd.read_csv('data/test/test_trd_b.csv')
print(train_df.shape)
print(test_df.shape)

(1367211, 8)
(142645, 7)


In [3]:
train_df.head(5)

,id,flag,Dat_Flg1_Cd,Dat_Flg3_Cd,Trx_Cod1_Cd,Trx_Cod2_Cd,trx_tm,cny_trx_amt
0,UFDC88A,0,B,B,1,116,2019-06-20 07:15:28,-127.99000
1,UFDC88A,0,B,B,1,116,2019-06-16 10:09:13,-55.88000
2,UFDC88A,0,B,B,1,136,2019-05-14 16:11:32,-557.00000
3,UFDC88A,0,B,B,1,136,2019-05-19 21:54:40,-77.80000
4,UFDC88A,0,B,B,1,113,2019-06-18 08:23:59,-271.62000


In [4]:
test_df.head(5)

,id,Dat_Flg1_Cd,Dat_Flg3_Cd,Trx_Cod1_Cd,Trx_Cod2_Cd,trx_tm,cny_trx_amt
0,U452CA2,B,B,1,130,2019-06-01 00:29:32,-5.00000
1,U452CA2,B,B,1,136,2019-05-15 00:00:00,-37.19000
2,U452CA2,C,B,3,309,2019-05-30 13:22:08,249.47000
3,U452CA2,B,B,1,108,2019-05-30 14:37:13,-29.94000
4,U452CA2,B,B,1,136,2019-06-21 00:00:00,-111.31000


In [5]:
train_df.describe(include='all')

,id,flag,Dat_Flg1_Cd,Dat_Flg3_Cd,Trx_Cod1_Cd,Trx_Cod2_Cd,trx_tm,cny_trx_amt
count,1367211,1367211.00000,1367211,1367211,1367211.00000,1367211.00000,1367211,1367211.00000
unique,31993,nan,2,3,nan,nan,1088223,nan
top,UBD9C48,nan,B,A,nan,nan,2019-06-01 00:29:49,nan
freq,5836,nan,1068119,695630,nan,nan,704,nan
mean,NaN,0.15735,NaN,NaN,1.47515,166.65003,NaN,4.89295
std,NaN,0.36413,NaN,NaN,0.77402,71.77708,NaN,69971.66252
min,NaN,0.00000,NaN,NaN,1.00000,101.00000,NaN,-23900000.00000
25%,NaN,0.00000,NaN,NaN,1.00000,117.00000,NaN,-200.00000
50%,NaN,0.00000,NaN,NaN,1.00000,134.00000,NaN,-24.00000
75%,NaN,0.00000,NaN,NaN,2.00000,209.00000,NaN,-2.00000


In [6]:
train_target = train_df['flag']
train_df.columns

Index(['id', 'flag', 'Dat_Flg1_Cd', 'Dat_Flg3_Cd', 'Trx_Cod1_Cd',
       'Trx_Cod2_Cd', 'trx_tm', 'cny_trx_amt'],
      dtype='object')

In [7]:
train_df.drop(['flag'], axis = 1, inplace=True)

In [8]:
train_df=pd.concat([train_df, test_df], axis=0, ignore_index=True)
train_df['cnt'] = 1
train_df.shape

(1509856, 8)

In [9]:
table = pd.pivot_table(train_df, values='cny_trx_amt', index=['id'],
                    columns=['Dat_Flg1_Cd'], aggfunc=np.sum)
table

Dat_Flg1_Cd,B,C
id,,
U000101,-7902.43000,7828.74000
U00015C,-56630.00000,27300.00000
U0001B8,-8840.98000,10535.97000
U00025B,-24606.87000,29908.88000
U000306,-217.00000,nan
...,...,...
UFFF6BB,-47756.75000,33789.56000
UFFF7F4,-91435.30000,202023.69000
UFFF9D8,-1996.00000,nan


In [10]:
table.columns = ['cny_trx_amt_B', 'cny_trx_amt_C']
table['trx_amt_balance'] = table['cny_trx_amt_B'] + table['cny_trx_amt_C']
table['trx_total_amount'] = table['cny_trx_amt_C'] - table['cny_trx_amt_B']
table['cny_trx_amt_B_percent'] = - table['cny_trx_amt_B'] / table['trx_total_amount']
table['cny_trx_amt_C_percent'] = table['cny_trx_amt_C'] / table['trx_total_amount']
table['id'] = table.index
table

,cny_trx_amt_B,cny_trx_amt_C,trx_amt_balance,trx_total_amount,cny_trx_amt_B_percent,cny_trx_amt_C_percent,id
id,,,,,,,
U000101,-7902.43000,7828.74000,-73.69000,15731.17000,0.50234,0.49766,U000101
U00015C,-56630.00000,27300.00000,-29330.00000,83930.00000,0.67473,0.32527,U00015C
U0001B8,-8840.98000,10535.97000,1694.99000,19376.95000,0.45626,0.54374,U0001B8
U00025B,-24606.87000,29908.88000,5302.01000,54515.75000,0.45137,0.54863,U00025B
U000306,-217.00000,nan,nan,nan,nan,nan,U000306
...,...,...,...,...,...,...,...
UFFF6BB,-47756.75000,33789.56000,-13967.19000,81546.31000,0.58564,0.41436,UFFF6BB
UFFF7F4,-91435.30000,202023.69000,110588.39000,293458.99000,0.31158,0.68842,UFFF7F4
UFFF9D8,-1996.00000,nan,nan,nan,nan,nan,UFFF9D8


In [11]:
table.to_csv('data/trd_amount.csv', index=False)

In [12]:
flg1_table = pd.pivot_table(train_df, values='cnt', index=['id'],
                    columns=['Dat_Flg1_Cd'], aggfunc=np.sum)

In [13]:
flg1_table.fillna(0, inplace=True)

In [14]:
flg1_table.columns = ['flg1_trd_cnt_B', 'flg1_trd_cnt_C']
flg1_table['trd_cnt'] = flg1_table['flg1_trd_cnt_B'] + flg1_table['flg1_trd_cnt_C']
flg1_table['flg1_trd_cnt_B_precent'] = flg1_table['flg1_trd_cnt_B'] / flg1_table['trd_cnt'] 
flg1_table['flg1_trd_cnt_C_precent'] = flg1_table['flg1_trd_cnt_C'] / flg1_table['trd_cnt']
flg1_table['id'] = flg1_table.index
flg1_table

,flg1_trd_cnt_B,flg1_trd_cnt_C,trd_cnt,flg1_trd_cnt_B_precent,flg1_trd_cnt_C_precent,id
id,,,,,,
U000101,38.00000,14.00000,52.00000,0.73077,0.26923,U000101
U00015C,8.00000,3.00000,11.00000,0.72727,0.27273,U00015C
U0001B8,34.00000,15.00000,49.00000,0.69388,0.30612,U0001B8
U00025B,37.00000,2.00000,39.00000,0.94872,0.05128,U00025B
U000306,4.00000,0.00000,4.00000,1.00000,0.00000,U000306
...,...,...,...,...,...,...
UFFF6BB,82.00000,71.00000,153.00000,0.53595,0.46405,UFFF6BB
UFFF7F4,159.00000,20.00000,179.00000,0.88827,0.11173,UFFF7F4
UFFF9D8,2.00000,0.00000,2.00000,1.00000,0.00000,UFFF9D8


In [15]:
flg1_table.to_csv('data/trd_cnt.csv', index=False)

In [16]:
flg3_table = pd.pivot_table(train_df, values='cnt', index=['id'],
                    columns=['Dat_Flg3_Cd'], aggfunc=np.sum)
flg3_table.columns = ['trx_flg3_A_cnt','trx_flg3_B_cnt','trx_flg3_C_cnt']
flg3_table

,trx_flg3_A_cnt,trx_flg3_B_cnt,trx_flg3_C_cnt
id,,,
U000101,17.00000,35.00000,nan
U00015C,nan,11.00000,nan
U0001B8,49.00000,nan,nan
U00025B,1.00000,38.00000,nan
U000306,nan,4.00000,nan
...,...,...,...
UFFF6BB,143.00000,10.00000,nan
UFFF7F4,54.00000,125.00000,nan
UFFF9D8,nan,2.00000,nan


In [17]:
flg3_table.fillna(0, inplace = True)
flg3_table['trd_flg3_cnt'] = flg3_table['trx_flg3_A_cnt'] + flg3_table['trx_flg3_B_cnt'] + flg3_table['trx_flg3_C_cnt']
flg3_table['trd_flg3_A_percent'] = flg3_table['trx_flg3_A_cnt'] / flg3_table['trd_flg3_cnt']
flg3_table['trd_flg3_B_percent'] = flg3_table['trx_flg3_B_cnt'] / flg3_table['trd_flg3_cnt']

In [18]:
flg3_table['trd_flg3_C_percent'] = flg3_table['trx_flg3_C_cnt'] / flg3_table['trd_flg3_cnt']

In [19]:
flg3_table

,trx_flg3_A_cnt,trx_flg3_B_cnt,trx_flg3_C_cnt,trd_flg3_cnt,trd_flg3_A_percent,trd_flg3_B_percent,trd_flg3_C_percent
id,,,,,,,
U000101,17.00000,35.00000,0.00000,52.00000,0.32692,0.67308,0.00000
U00015C,0.00000,11.00000,0.00000,11.00000,0.00000,1.00000,0.00000
U0001B8,49.00000,0.00000,0.00000,49.00000,1.00000,0.00000,0.00000
U00025B,1.00000,38.00000,0.00000,39.00000,0.02564,0.97436,0.00000
U000306,0.00000,4.00000,0.00000,4.00000,0.00000,1.00000,0.00000
...,...,...,...,...,...,...,...
UFFF6BB,143.00000,10.00000,0.00000,153.00000,0.93464,0.06536,0.00000
UFFF7F4,54.00000,125.00000,0.00000,179.00000,0.30168,0.69832,0.00000
UFFF9D8,0.00000,2.00000,0.00000,2.00000,0.00000,1.00000,0.00000


In [20]:
flg3_table['id'] = flg3_table.index
flg3_table.to_csv('data/trd_flg3_cnt.csv', index=False)

In [21]:
cod1_table = pd.pivot_table(train_df, values='cnt', index=['id'],
                    columns=['Trx_Cod1_Cd'], aggfunc=np.sum)

cod1_table.columns = ['trx_cod1_1_cnt','trx_cod1_2_cnt','trx_cod1_3_cnt']
cod1_table

,trx_cod1_1_cnt,trx_cod1_2_cnt,trx_cod1_3_cnt
id,,,
U000101,37.00000,11.00000,4.00000
U00015C,8.00000,nan,3.00000
U0001B8,31.00000,12.00000,6.00000
U00025B,35.00000,nan,4.00000
U000306,2.00000,nan,2.00000
...,...,...,...
UFFF6BB,79.00000,57.00000,17.00000
UFFF7F4,155.00000,16.00000,8.00000
UFFF9D8,2.00000,nan,nan


In [22]:
cod1_table['trx_cod1_cnt'] = cod1_table['trx_cod1_1_cnt'] + cod1_table['trx_cod1_2_cnt'] +  cod1_table['trx_cod1_3_cnt']
cod1_table['trx_cod1_cnt_1_percent'] = cod1_table['trx_cod1_1_cnt'] / cod1_table['trx_cod1_cnt']
cod1_table['trx_cod1_cnt_2_percent'] = cod1_table['trx_cod1_2_cnt'] / cod1_table['trx_cod1_cnt']
cod1_table['trx_cod1_cnt_3_percent'] = cod1_table['trx_cod1_3_cnt'] / cod1_table['trx_cod1_cnt']
cod1_table

,trx_cod1_1_cnt,trx_cod1_2_cnt,trx_cod1_3_cnt,trx_cod1_cnt,trx_cod1_cnt_1_percent,trx_cod1_cnt_2_percent,trx_cod1_cnt_3_percent
id,,,,,,,
U000101,37.00000,11.00000,4.00000,52.00000,0.71154,0.21154,0.07692
U00015C,8.00000,nan,3.00000,nan,nan,nan,nan
U0001B8,31.00000,12.00000,6.00000,49.00000,0.63265,0.24490,0.12245
U00025B,35.00000,nan,4.00000,nan,nan,nan,nan
U000306,2.00000,nan,2.00000,nan,nan,nan,nan
...,...,...,...,...,...,...,...
UFFF6BB,79.00000,57.00000,17.00000,153.00000,0.51634,0.37255,0.11111
UFFF7F4,155.00000,16.00000,8.00000,179.00000,0.86592,0.08939,0.04469
UFFF9D8,2.00000,nan,nan,nan,nan,nan,nan


In [23]:
cod1_table['id'] = cod1_table.index
cod1_table.to_csv('data/trd_cod1_cnt.csv', index=False)

In [24]:
cod2_table = pd.pivot_table(train_df, values='cnt', index=['id'],
                    columns=['Trx_Cod2_Cd'], aggfunc=np.sum)

# cod1_table.columns = ['trx_cod1_1_cnt','trx_cod1_2_cnt','trx_cod1_3_cnt']
cod2_table

Trx_Cod2_Cd,101,102,103,104,105,106,107,108,109,110,...,302,303,304,305,306,307,308,309,310,311
id,,,,,,,,,,,,,,,,,,,,,
U000101,nan,1.00000,22.00000,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,4.00000,nan,nan
U00015C,nan,nan,1.00000,nan,nan,nan,nan,nan,2.00000,3.00000,...,nan,nan,nan,nan,nan,nan,nan,3.00000,nan,nan
U0001B8,nan,3.00000,nan,nan,nan,nan,nan,nan,nan,nan,...,2.00000,nan,nan,nan,nan,nan,4.00000,nan,nan,nan
U00025B,2.00000,nan,nan,3.00000,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,2.00000,2.00000,nan
U000306,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,2.00000,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
UFFF6BB,2.00000,nan,1.00000,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,1.00000,11.00000,5.00000,nan,nan
UFFF7F4,2.00000,6.00000,17.00000,2.00000,nan,nan,2.00000,6.00000,nan,nan,...,nan,nan,nan,nan,nan,2.00000,nan,4.00000,2.00000,nan
UFFF9D8,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


In [25]:
cod2_table['max_cnt_trx_cod2'] = cod2_table.idxmax(axis=1, skipna=True) 
cod2_table

Trx_Cod2_Cd,101,102,103,104,105,106,107,108,109,110,...,303,304,305,306,307,308,309,310,311,max_cnt_trx_cod2
id,,,,,,,,,,,,,,,,,,,,,
U000101,nan,1.00000,22.00000,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,4.00000,nan,nan,103
U00015C,nan,nan,1.00000,nan,nan,nan,nan,nan,2.00000,3.00000,...,nan,nan,nan,nan,nan,nan,3.00000,nan,nan,110
U0001B8,nan,3.00000,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,4.00000,nan,nan,nan,132
U00025B,2.00000,nan,nan,3.00000,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,2.00000,2.00000,nan,136
U000306,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,2.00000,nan,134
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
UFFF6BB,2.00000,nan,1.00000,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,1.00000,11.00000,5.00000,nan,nan,128
UFFF7F4,2.00000,6.00000,17.00000,2.00000,nan,nan,2.00000,6.00000,nan,nan,...,nan,nan,nan,nan,2.00000,nan,4.00000,2.00000,nan,132
UFFF9D8,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,118


In [26]:
cod2_table['id'] = cod2_table.index

In [27]:
cod2_table = pd.DataFrame({'id':cod2_table['id'], 'max_cnt_trx_cod2': cod2_table['max_cnt_trx_cod2']})
cod2_table.fillna('None', inplace=True)
cod2_table.isnull().sum()

id                  0
max_cnt_trx_cod2    0
dtype: int64

In [28]:
cod2_table.to_csv('data/trd_max_cnt_cod2.csv', index=False)

In [29]:
cod2_table.max_cnt_trx_cod2.isnull().sum()

0